In [ ]:
system"cd ",getenv[`HOME],"/course-introductory-workshop"
.trn.nbdir:system"cd"
\l scripts/loaddata.q

**Learning objective**
* How to apply a left join
* How to apply an as-of join

# Joins

A join combines data from two tables, or from a table and a dictionary.

Some joins are keyed, in that columns in the first argument are matched with the key columns of the second argument.

Some joins are as-of, where a time column in the first argument specifies corresponding intervals in a time column of the second argument. Such joins are not keyed.

In each case, the result has the merge of columns from both arguments. Where necessary, rows are filled with nulls or zeros. 

Here is a list of some joins possible using kdb+/q:

+ [Left Join](https://code.kx.com/q/ref/lj/)
+ [AJ (As-of) Join](https://code.kx.com/q/ref/aj/)

In order to demonstrate some common join types we will use data from preloaded table, `weather`,  which corresponds to the same time as our taxi data. In this case we will load some weather data and try and get some insight in how this affected taxi journeys around this period.

In [ ]:
// Check number of records in weather and the meta
count weather
meta weather

In [ ]:
select mindate:min date, maxdate:max date from weather

##### Exercise 9
- Display the max and min temperatures for NYC each week through January (For this query a week is just every 7 days)

In [ ]:
select max maxtemp, min mintemp by 7 xbar date from weather
//alternative way- select max maxtemp, min mintemp by date.week from weather

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> Note the difference in start date in the 2 solutions. In kdb+/q when using 7 xbar the date jumps from one Saturday to the next Saturday. This is because kdb+ follows the J2000 international standard starting from 2000.01.01 which happens to be a Saturday. When using date.week the date starts from a Monday as this is the first day of a week in kdb+.  </i></p>

In [ ]:
//Enter your code here 

In [ ]:
ex9[] //check correct output

We now have two tables of related data, taxi trips and weather data, for each day on which a trip occurred.

It would be nice to combine these tables so we could easily ask questions across both data sets.

For example, are average trip durations shorter or longer on days with lots of precipitation?

## Left Join

Like other SQL languages, qSQL supports a number of join operations. Here we will use a left join to create a single table containing both trip and weather data. In kdb+/q the `lj` operator requires one or more common columns between the tables to join across. 

`t1 lj t2` - [left join](https://code.kx.com/q/ref/lj/)

<img src="images/LeftJoin.png" width="400" height="200">

For our purpose we will be looking at the daily weather data so we'll use the `date` column, which is in both tables.

In [ ]:
// Find the number of trips per day

jan09:select from trips where date within 2009.01.01 2009.01.31
jan09C:select trips: count i by date from jan09
jan09C

Looking more closely at `jan09C`, it doesn't look like a normal kdb+ table as there is a vertical line appearing between the columns `date` and `x`. This informs us that this table is actually a **keyed table** which we require to have in most kdb+ joins. But how do we create keyed tables explicitly ? Well, we have two choices:

1. Using the syntax above - we can use a by clause 
2. Using [xkey](https://code.kx.com/q/ref/keys/#xkey) or [!(bang)](https://code.kx.com/q/ref/enkey/)

In [ ]:
`date xkey weather //we are keying on date 
1!weather          //we are keying on the first column 
3!weather          //we can key on N number of columns

If we want to unkey a keyed table using the `!`: 

In [ ]:
0!jan09C     

The `lj` operator requires that at least the right hand table argument be keyed. A table can be keyed in a number of ways, however the easiest is to use the [`xkey`](https://code.kx.com/q/ref/keys/#xkey) function

In [ ]:
// select date and precipitation from the weather table
// key the result on date
// join to the unkeyed table jan09C (0! unkeys the table)
jan09W:jan09C lj `date xkey select date, precip from weather 
jan09W

jan09W:jan09C lj select avg precip by date from weather //using the by clause to key

Let's check will we get the same result if the left hand table is unkeyed?

In [ ]:
unkeyedJan09C:0!jan09C
unkeyedJan09C lj `date xkey select date, precip from weather

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> The left-hand table can be keyed or unkeyed. The format of the left-hand table will dictate the format of the result table. </i></p> 

Now we can look at trips vs precipitation

In [ ]:
select date,trips,precip from jan09W

##### Exercise 10
- Create a new join which joins the number of trips with the average temperature from the weather data, per day for the month of January

In [ ]:
jan09C lj `date xkey select date, avgtemp from weather 

In [ ]:
//Enter your code here 

In [ ]:
ex10[] //check correct output

## As-of Join

`aj[matching columns;t1;t2]` - [aj join](https://code.kx.com/q/ref/aj/)

qSQL also supports time-series joins, a powerful feature not typically found in other databases and languages.

Given the data we have, we could ask what were the latest pick-ups for each vendor, as of a particular time.

We will create a temporary time table with a minimum date time for each vendor:

Let's say there are three reports of individuals who have lost their phone or wallet who were picked up shortly before the time who said how many passengers were in the taxi. Which vendor were they riding with?

In [ ]:
timetab:([] passengers:1 2 3; event_time:2009.01.06D03:30:00+00:30*til 3)
timetab

Using `aj`, we can look up the table `jan09` to find out what was the last trip taken at each of the times above with those passengers:

In [ ]:
aj[`passengers`event_time;timetab;select passengers, event_time:pickup_time, vendor, pickup_time from jan09]

The result is the record for each vendor with the event_time ≤ to the time we specified.
- An `aj` join will always select the last record before the specified time.

In [ ]:
timetab:([] passengers:1 2 3 4 5 6; event_time:2009.01.06D03:30:00+00:30*til 6)
timetab

We have created a new timetab table. What will the output as-of join be now?

In [ ]:
aj[`passengers`event_time;timetab;select passengers, event_time:pickup_time, vendor, pickup_time from jan09]

##### Exercise 11

Find the latest trips as of 09:30 on the 31st of January for each vendor.

In [ ]:
timetab:([] vendor: `VTS`DDS`CMT; pickup_time:3#2009.01.31D09:30:00)
aj[`vendor`pickup_time;timetab;jan09]

In [ ]:
//Enter your code here

In [ ]:
ex11[] //check correct output